<a href="https://colab.research.google.com/github/yuuuchen/ProgressPal/blob/main/0815_%E4%B8%BB%E7%A8%8B%E5%BC%8F_%E7%89%88%E6%9C%AC%E4%B8%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: google-genai
    Found existing installation: google-genai 1.29.0
    Uninstalling google-genai-1.29.0:
      Successfully uninstalled google-genai-1.29.0


In [1]:
from google import genai
from google.genai import types
from google.colab import userdata
from IPython.display import Markdown
import textwrap, os, json, re

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def extract_json(text: str):
    """容錯：去掉 ``` 與多餘文字，抓出最外層 JSON 陣列"""
    if not text:
        raise ValueError("模型回傳是空字串")
    t = text.strip()
    if t.startswith("```"):
        t = re.sub(r"^```(?:json)?\s*|\s*```$", "", t, flags=re.DOTALL)
    try:
        return json.loads(t)
    except json.JSONDecodeError:
        m = re.search(r"\[.*\]", t, flags=re.DOTALL)
        if m:
            return json.loads(m.group(0))
        raise

# 設定 API 金鑰
API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=API_KEY)

教材切分



In [3]:
# 上傳 PDF
pdf_path = "串列.pdf"
assert os.path.exists(pdf_path), f"找不到檔案: {pdf_path}"

myfile = client.files.upload(file=pdf_path)
print("已上傳:", myfile.uri)

# Gemini 分段
split_prompt = """
你是一位資料結構課程的助教。請將這份 PDF（視為同一章節）劃分為若干「單元」。
規則：
- 依教材自然結構切分，每單元內容與名稱皆不重疊。
- 每個單元用 1 行「unit_title」命名（不超過 16 個字，務必有語意）。
- 「content」為該單元需教授的重點摘要（包含定義/重點清單/舉例），長度建議 300~500 字。
- 僅輸出 JSON，勿加任何說明或標註。
"""

gen_config = types.GenerateContentConfig(
    response_mime_type="application/json",
    response_schema=types.Schema(
        type=types.Type.ARRAY,
        items=types.Schema(
            type=types.Type.OBJECT,
            properties={
                "unit_title": types.Schema(type=types.Type.STRING),
                "content": types.Schema(type=types.Type.STRING),
            },
            required=["unit_title", "content"],
        ),
    ),
)

split_response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[types.Content(role="user", parts=[
        types.Part(file_data=types.FileData(
            file_uri=myfile.uri,
            mime_type="application/pdf"
        )),
        types.Part(text=split_prompt)
    ])],
    config=gen_config
)

raw_text = split_response.text

try:
    units = json.loads(raw_text)
except json.JSONDecodeError:
    units = extract_json(raw_text)

assert isinstance(units, list) and len(units) > 0, "分段結果為空"
for i, u in enumerate(units, 1):
    print(f"{i:02d}. {u['unit_title']}  (約 {len(u['content'])} 字)")

已上傳: https://generativelanguage.googleapis.com/v1beta/files/vklnrgb2zef9
01. 串列表示法  (約 250 字)
02. 一維串列  (約 411 字)
03. 二維串列  (約 444 字)
04. 三維串列與n維串列  (約 442 字)
05. Python 語言的串列表示法  (約 279 字)
06. 矩陣相乘  (約 142 字)
07. 稀疏矩陣  (約 191 字)


**單元教學**

無特定說明教材呈現方式與學生程度(初學/複習)

In [ ]:
# 固定教學指令
system_instruction = "你是一位資料結構課程的家教，請用繁體中文仔細地講解以下單元內容，並適度舉例(需提供解答)。"

# 教學情緒對應表（回傳 tone 和 style）
def emotion_instruction_map(emotion):
    mapping = {
        "frustrated": {"tone": "溫暖且安撫", "style": "循序漸進、拆解問題"},
        "confused": {"tone": "溫和且耐心", "style": "舉例對照、比喻解釋"},
        "bored": {"tone": "活潑且有趣", "style": "加入情境化案例、互動提問"},
        "engaged": {"tone": "積極且肯定", "style": "深入探討、引導延伸思考"},
        "surprised": {"tone": "熱情且鼓勵", "style": "延伸趣味點、引入新視角"},
        "joyful": {"tone": "輕鬆且正向", "style": "融入挑戰題、鼓勵自我探索"},
    }
    return mapping.get(emotion, {"tone": "中性", "style": "一般解釋"})

conversation_history = ""
last_emotion_profile = {"tone": "中性", "style": "一般解釋"}  # 第一單元預設中性
last_emotion = "中性"

for idx, unit in enumerate(units, start=1):
    print(f"\n=== 單元 {idx}：{unit['unit_title']} ===\n")

    # 教學
    lecture_prompt = (
        system_instruction
        + f"\n你是一位{last_emotion_profile['tone']}的資料結構助教，現在面對一位感到{last_emotion}的學生"
          f"請用{last_emotion_profile['style']}方式講解(。\n"
        + unit["content"]
    )
    print("🔹 正在生成教學內容...")
    lecture_resp = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=[types.Content(role="user", parts=[
            types.Part(text=lecture_prompt)
        ])]
    )
    lecture_text = lecture_resp.text
    display(to_markdown(lecture_text))

    print("🔹 進入問答階段")
    # 問答階段
    while True:
        q = input("學生提問或輸入 next 進入下一單元：")
        if q.strip().lower() == "next":
            break

        tone = last_emotion_profile.get("tone", "中性")
        style = last_emotion_profile.get("style", "一般解釋")

        answer_prompt = f"""你是一位{tone}的資料結構助教。
        請用{style}方式，面對一位感到{last_emotion}的學生，
        回答以下問題：{q}
        請根據以下教材回答，避免超出範圍：
        {unit['content']}
        若問題與教材無關，請回覆「這個問題與本單元教材無關」。
        """
        print("🔹 正在生成答覆...")
        ans_resp = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=[types.Content(role="user", parts=[
                types.Part(text=answer_prompt)
            ])]
        )
        ans_text = ans_resp.text
        display(to_markdown(ans_text))
        conversation_history += f"\n[單元{idx}] 問：{q}\n答：{ans_text}"

    # 問答結束後，輸入新的情緒（第二單元開始才更新）
    detected_emotion = input("請輸入偵測到的學生情緒：")
    last_emotion_profile = emotion_instruction_map(detected_emotion)
    last_emotion = detected_emotion


=== 單元 1：串列表示法 ===

🔹 正在生成教學內容...


> 哈囉！別擔心，線性串列其實沒有想像中那麼可怕，我們來一步一步拆解它。
> 
> **什麼是線性串列？**
> 
> 簡單來說，線性串列就是一種排列資料的方式，就像一排隊伍一樣。隊伍中的每個人（也就是資料）都有一個特定的位置，這個位置就決定了它的順序。想像一下你在排隊買飲料，你的位置決定了你什麼時候可以點餐。
> 
> 更正式的說法，線性串列的特性是：
> 
> *   **線性排列：** 資料是按照一個固定的順序排列的，每個資料都有一個明確的前後關係。
> *   **位置決定順序：** 資料在串列中的位置（index）決定了它在整個串列中的順序。
> 
> 線性串列有很多別名，像是循序串列、有序串列，其實指的都是同一件事。
> 
> **線性串列的常見操作**
> 
> 就像隊伍有不同的狀況，線性串列也需要進行一些常見的操作，來維持隊伍的秩序：
> 
> 1.  **取出第 i 項資料：** 就像點名一樣，我們要能夠找到隊伍中特定位置的人。
>     *   **例子：** 在 `["A", "B", "C", "D", "E"]` 這個串列中，取出第 3 項資料（注意索引從 0 開始），結果就是 "C"。
> 2.  **計算串列長度：** 知道隊伍有多長，才能安排後續的活動。
>     *   **例子：** `["A", "B", "C", "D", "E"]` 這個串列的長度是 5。
> 3.  **由左至右或由右至左取出資料：** 就像從隊伍頭或隊伍尾開始依序叫號。
>     *   **例子：** 從左至右取出 `["A", "B", "C"]` 的資料，順序就是 "A"、"B"、"C"。
> 4.  **在第 i 項加入一個新值：** 就像有人插隊，插入後面的所有人都要往後移。
>     *   **例子：** 在 `["A", "B", "C", "D"]` 的第 2 項（索引為 1）加入 "X"，結果會變成 `["A", "X", "B", "C", "D"]`。
> 5.  **刪除第 i 項：** 就像有人離開隊伍，後面的人都要往前補上。
>     *   **例子：** 刪除 `["A", "B", "C", "D"]` 的第 3 項（索引為 2），結果會變成 `["A", "B", "D"]`。
> 
> **Python 中的線性串列**
> 
> 在 Python 裡面，我們通常使用 `list` (串列) 來表示線性串列。Python 的 `list` 提供了很多方便的方法來進行上面提到的操作。
> 
> **重要提醒：Python 的索引從 0 開始！**
> 
> 這是很多初學者容易犯錯的地方。在 Python 中，串列的第一個元素的索引是 0，第二個是 1，依此類推。
> 
> **實例示範 (Python)**
> 
> ```python
> # 建立一個線性串列
> my_list = ["A", "B", "C", "D", "E"]
> 
> # 1. 取出第 3 項資料
> print(my_list[2])  # 輸出：C
> 
> # 2. 計算串列長度
> print(len(my_list))  # 輸出：5
> 
> # 3. 在第 2 項加入一個新值 "X"
> my_list.insert(1, "X")  # 在索引 1 的位置插入 "X"
> print(my_list)  # 輸出：['A', 'X', 'B', 'C', 'D', 'E']
> 
> # 4. 刪除第 3 項
> del my_list[2]  # 刪除索引 2 的元素
> print(my_list)  # 輸出：['A', 'X', 'C', 'D', 'E']
> ```
> 
> **總結**
> 
> 線性串列是一種簡單但非常重要的資料結構。理解它的概念和操作，對於學習更複雜的資料結構和演算法都很有幫助。  最重要的是記住 Python 索引從 0 開始。
> 
> 希望這樣的解釋能幫助你更好地理解線性串列。如果你還有其他問題，隨時都可以問我！


🔹 進入問答階段
學生提問或輸入 next 進入下一單元：next
請輸入偵測到的學生情緒：joyful

=== 單元 2：一維串列 ===

🔹 正在生成教學內容...


> 哈囉！這位 Joyful 的同學，看到你這麼開心學習資料結構，我也覺得很棒！🎉 讓我們一起來探索串列位址計算的奧秘吧！😎
> 
> 你說的這些公式，其實就是串列在記憶體中是如何排列的線索。就像我們在排隊買飲料，每個人佔一定的空間，要知道第 i 個人站在哪裡，就需要知道隊伍的起點和每個人佔的空間。
> 
> **核心概念：**
> 
> *   **起始位址 (a0):** 就像隊伍的第一個人站在哪裡。
> *   **元素大小 (d):** 就像每個人佔的空間大小。
> *   **索引 (i):** 就像你要找的是隊伍中的第幾個人。
> *   **索引範圍 (t:u):** 就像隊伍是從第 t 個人排到第 u 個人。
> 
> **公式拆解：**
> 
> 1.  **A(0:u-1) 的情況:** `A(i) = a0 + i*d`
>     *   這表示串列從索引 0 開始，第 i 個元素的位址就是起始位址加上 i 個元素大小。
>     *   **舉例：** 假設我們有一個整數串列 A(0:9)，起始位址是 1000，每個整數佔 4 個位元組，那麼 A(3) 的位址就是 1000 + 3 * 4 = 1012。
> 
> 2.  **A(t:u) 的情況:** `A(i) = a0 + (i−t)*d`
>     *   這表示串列從索引 t 開始，第 i 個元素的位址需要考慮起始索引 t 的偏移量。
>     *   **舉例：** 假設我們有一個浮點數串列 A(5:15)，起始位址是 2000，每個浮點數佔 8 個位元組，那麼 A(8) 的位址就是 2000 + (8 - 5) * 8 = 2024。
> 
> **挑戰題來囉！** 🚀
> 
> 有一位喜歡挑戰的同學想設計一個更有效率的資料結構，他想把二維陣列壓平成一維陣列來儲存。假設有一個二維陣列 `B(1:5, 1:10)`，每個元素佔用 2 個位元組，且 `B(1,1)` 的位址是 500。請問 `B(3,7)` 的位址是多少？ (提示：你需要先思考二維陣列如何壓平成一維陣列，以及列優先和行優先的差別！)
> 
> **解答：**
> 
> *   **假設採用列優先的儲存方式：** 也就是先存完第一列，再存第二列，以此類推。
> *   `B(3,7)` 之前的元素個數： (3-1) * 10 + (7-1) = 20 + 6 = 26
> *   `B(3,7)` 的位址： 500 + 26 * 2 = 552
> 
> **鼓勵自我探索：**
> 
> *   **試著用不同的程式語言 (例如 Python, C++) 實作這些公式，並驗證結果。**
> *   **思考一下，如果串列的元素大小不固定 (例如儲存字串)，這些公式該如何修改？**
> *   **除了列優先，還有行優先的儲存方式。試著用行優先的方式解上面的挑戰題。**
> 
> 資料結構就像積木，這些基本的概念是 building block，學好它們才能蓋出更酷的城堡！💪 不要怕犯錯，每一次的嘗試都是學習的機會。加油！期待你蓋出屬於你的資料結構城堡！🏰


🔹 進入問答階段
學生提問或輸入 next 進入下一單元：next
請輸入偵測到的學生情緒：confused

=== 單元 3：二維串列 ===

🔹 正在生成教學內容...


...

> 哈囉！這位同學，看你好像有點提不起勁，沒關係，資料結構有時候的確會讓人覺得有點... 嗯... 乾澀 (笑)。 但別擔心，今天我們要講的「n維串列的表示方式」，其實就像在玩空間規劃遊戲，搞懂了就覺得超級有趣！
> 
> 準備好跟我一起蓋房子、玩收納了嗎？ Let's go!
> 
> **情境導入：想像一下，我們是超強收納大師！**
> 
> 你是一位收納大師，負責管理一個超級巨大的倉庫。這個倉庫不是平面的，而是有 **n 層** 的立體結構！每一層又分成好幾個隔間，我們要怎麼有效地找到並存放每個貨物呢？ 這就是 n 維串列要解決的問題啦！
> 
> **什麼是 n 維串列？**
> 
> 簡單來說，n 維串列就像一個多層次的表格或矩陣。
> 
> *   **1 維串列:** 就像一條直線，上面有很多格子可以放東西。 比如 `[蘋果, 香蕉, 橘子]`
> *   **2 維串列:** 就像一個表格，有行和列。 比如一個放學生分數的表格：
>     ```
>     [
>         [90, 85, 88],  # 小明的國、英、數成績
>         [78, 92, 80],  # 小美的國、英、數成績
>         [85, 88, 95]   # 小剛的國、英、數成績
>     ]
>     ```
> *   **3 維串列:** 就像一個魔術方塊，不只可以上下左右看，還可以前後看。
> *   **n 維串列:** 想像一下無限延伸的魔術方塊，就是 n 維串列啦！ (當然，超過三維就比較難在腦袋裡直接想像了，但概念是一樣的！)
> 
> **現在，重點來了！ 我們要怎麼在這麼複雜的倉庫 (n 維串列) 裡找到特定的貨物呢？**
> 
> 這時候就要用到兩種不同的「尋址方式」：
> 
> 1.  **以列為主 (Row-Major Order)：**
> 2.  **以行為主 (Column-Major Order)：**
> 
> **1. 以列為主 (Row-Major Order)： 就像你讀書一樣，一行一行的讀**
> 
> 想像一下，我們在讀一本書。你會怎麼讀？ 一行一行從左到右讀，讀完一行再讀下一行。 以列為主就是這個概念！ 在記憶體裡，我們會先把第一列的元素放完，再放第二列，依此類推。
> 
> *   **公式:** `A(i1, i2, i3, …, in) = a0 + ∑ im * am`
>     *   `A(i1, i2, i3, …, in)`:  你要找的貨物 (元素) 在倉庫裡的位置。`i1, i2, i3, …, in` 分別代表每一維的索引值(index)。
>     *   `a0`: 倉庫的第一個位置，就像書的第一頁。
>     *   `∑ im * am`:  每一維的索引值乘以一個權重，然後加總。 這個權重決定了每一維在記憶體裡佔多少空間。
>         *   m 從 1 開始，代表第一維
>         *   `u`: 每一維的最大索引值
> 
> **舉例說明 (2 維串列):**
> 
> 假設我們有一個 2 維串列 `A(0:2, 0:3)`，也就是一個 3 行 4 列的表格。 我們想找到 `A(1, 2)` 這個位置的元素。 假設 `a0 = 1000` (也就是 A(0,0) 的記憶體位置) 。 我們用以列為主的公式來計算：
> 
> `A(i1, i2) = a0 + i1 * u2 + i2`
> 
> *   `i1 = 1` (第二列)
> *   `i2 = 2` (第三行)
> *   `u2 = 4` (每一列有 4 個元素)
> 
> 所以，`A(1, 2) = 1000 + 1 * 4 + 2 = 1006`
> 
> 這表示 `A(1, 2)` 這個元素在記憶體中的位置是 1006。
> 
> **解題技巧**
> 
> n 維陣列以列為主在求記憶體位置時，最右邊的維度變化最快。
> 
> **2. 以行為主 (Column-Major Order): 就像看報紙一樣，一欄一欄的看**
> 
> 想像一下，你在看報紙。你會怎麼看？  一欄一欄從上到下看，看完一欄再看下一欄。 以行為主就是這個概念！ 在記憶體裡，我們會先把第一行的元素放完，再放第二行，依此類推。
> 
> *   **公式:** `A(i1, i2, i3, ………, in) = a0 + ∑ im* am`
>     *   `A(i1, i2, i3, …, in)`: 你要找的貨物 (元素) 在倉庫裡的位置。
>     *   `a0`: 倉庫的第一個位置。
>     *   `∑ im * am`: 每一維的索引值乘以一個權重，然後加總。
> 
> **舉例說明 (2 維串列):**
> 
> 假設我們有一個 2 維串列 `A(0:2, 0:3)`，也就是一個 3 行 4 列的表格。 我們想找到 `A(1, 2)` 這個位置的元素。 假設 `a0 = 1000` (也就是 A(0,0) 的記憶體位置) 。 我們用以行為主的公式來計算：
> 
> `A(i1, i2) = a0 + i1 + i2 * u1`
> 
> *   `i1 = 1` (第二列)
> *   `i2 = 2` (第三行)
> *   `u1 = 3` (每一列有 3 個元素)
> 
> 所以，`A(1, 2) = 1000 + 1 + 2 * 3 = 1007`
> 
> 這表示 `A(1, 2)` 這個元素在記憶體中的位置是 1007。
> 
> **解題技巧**
> 
> n 維陣列以行為主在求記憶體位置時，最左邊的維度變化最快。
> 
> **互動提問：**
> 
> 1.  如果我們要把一個 3 維的魔術方塊 (例如 `A(0:2, 0:2, 0:2)`) 放到記憶體裡，以列為主和以行為主，哪一種方式比較直覺？ 為什麼？ (提示：想想你怎麼一層一層堆疊魔術方塊)
> 2.  在實際應用中，什麼樣的情況下我們會選擇以列為主，什麼樣的情況下我們會選擇以行為主？ (提示：不同的程式語言或硬體架構可能有不同的偏好)
> 
> **重點整理：**
> 
> *   n 維串列就像多層次的倉庫，用來存放資料。
> *   以列為主就像讀書，一行一行存放。
> *   以行為主就像看報紙，一欄一欄存放。
> *   不同的尋址方式會影響資料在記憶體中的位置。
> 
> 希望透過這個收納大師的比喻，你能更輕鬆地理解 n 維串列的表示方式！ 其實資料結構就像工具，了解它們的特性，才能更有效地解決問題。
> 
> 現在，深呼吸一下，休息一下，如果還有任何問題，隨時都可以問我喔！ 我們一起把資料結構變成有趣的冒險吧！


🔹 進入問答階段
學生提問或輸入 next 進入下一單元：next
請輸入偵測到的學生情緒：joyful

=== 單元 6：Python 串列表示法 ===

🔹 正在生成教學內容...


> 哈囉！這位 Joyful 的同學，看到你這麼開心學習資料結構，我也覺得非常興奮！今天我們要來挑戰多維串列，準備好一起探索了嗎？
> 
> **複習一下：一維、二維、三維串列的概念**
> 
> *   **一維串列：** 就像一條直線，每個元素都有一個位置 (索引)。例如：`A = [0] * 20` 創造了一個有 20 個元素的串列，每個元素預設都是 0。你可以想像成一排置物櫃，每個置物櫃都有一個編號（從 0 開始）。
> *   **二維串列：** 就像一張表格，有列 (rows) 和行 (columns)。例如：`A = [[0] * 10 for rows in range(20)]` 創造了一個 20 列、10 行的表格，每個格子預設都是 0。你可以想像成一個教室，有 20 排座位，每排有 10 個位子。
> *   **三維串列：** 就像一個立體的魔術方塊，有深度 (depth)、列 (rows) 和行 (columns)。想像一下有很多張二維表格疊在一起，就形成了三維串列。
> 
> **挑戰題：解讀程式碼片段**
> 
> 現在，我們來看看你提供的程式碼片段（請提供程式碼，我才能更精準地解說喔！），我會一步一步帶你解讀，讓你不再害怕多維串列！
> 
> **解題步驟：**
> 
> 1.  **了解程式碼的功能：** 首先，我們要搞清楚這段程式碼想要做什麼。它是在創造一個三維串列嗎？還是在對現有的串列進行操作？仔細閱讀每一行程式碼，找出它的目的。
> 
> 2.  **追蹤變數的變化：** 程式碼中可能會有一些變數，它們的值會隨著程式的執行而改變。我們要像偵探一樣，追蹤這些變數的變化，才能了解程式的運作流程。
> 
> 3.  **模擬執行：** 就像在腦海中播放電影一樣，我們要一步一步地模擬程式的執行過程。特別注意迴圈的執行次數和條件判斷，這些都會影響最終的結果。
> 
> 4.  **預測輸出結果：** 根據我們的理解和模擬，預測程式碼的輸出結果。
> 
> **範例程式碼與解說 (假設程式碼如下):**
> 
> ```python
> A = [[[0] * 3 for _ in range(4)] for _ in range(2)]
> 
> for i in range(2):
>     for j in range(4):
>         for k in range(3):
>             A[i][j][k] = i + j + k
> 
> print(A[1][2][1])
> ```
> 
> **解說:**
> 
> 1.  **宣告三維陣列:** `A = [[[0] * 3 for _ in range(4)] for _ in range(2)]`
>     *   這段程式碼宣告了一個名為 `A` 的三維陣列。
>     *   這個陣列的結構是 2 x 4 x 3，也就是說，它有 2 個「層」、每一層有 4 列、每一列有 3 個元素。
>     *   所有元素初始值都是 0。
> 
> 2.  **巢狀迴圈賦值:**
> 
>     ```python
>     for i in range(2): # 0, 1
>         for j in range(4): # 0, 1, 2, 3
>             for k in range(3): # 0, 1, 2
>                 A[i][j][k] = i + j + k
>     ```
> 
>     *   這段程式碼使用三個巢狀迴圈來遍歷三維陣列 `A` 的每一個元素。
>     *   `i` 代表層數，範圍是 0 到 1。
>     *   `j` 代表列數，範圍是 0 到 3。
>     *   `k` 代表每列中的元素，範圍是 0 到 2。
>     *   在迴圈內部，將 `A[i][j][k]` 的值設定為 `i + j + k`，也就是三個索引值的總和。
> 
> 3.  **輸出:** `print(A[1][2][1])`
>     *   這行程式碼會印出 `A[1][2][1]` 這個元素的值。
>     *   根據前面的賦值邏輯，`A[1][2][1] = 1 + 2 + 1 = 4`。
> 
> **解答:**
> 
> 因此，這個程式碼片段的輸出結果是 **4**。
> 
> **自我探索與挑戰：**
> 
> 1.  **修改程式碼：** 試著修改迴圈中的賦值公式，例如改成 `A[i][j][k] = i * j * k`，看看輸出結果會變成什麼？為什麼？
> 2.  **擴展維度：** 挑戰一下，能不能寫出程式碼來建立一個四維串列？
> 3.  **應用場景：** 想想看，多維串列可以用在哪些實際的應用場景中？例如，影像處理、遊戲地圖等等。
> 
> **溫馨提示：**
> 
> *   多維串列可能會讓人覺得有點複雜，但只要一步一步拆解，你會發現它其實很有趣！
> *   不要害怕犯錯，從錯誤中學習是成長的最好方式！
> *   如果你在學習過程中遇到任何問題，隨時都可以問我喔！我很樂意幫助你！
> 
> 加油！我相信你一定可以掌握多維串列的奧秘！期待你分享你的學習心得和挑戰成果喔！


🔹 進入問答階段


KeyboardInterrupt: Interrupted by user

有指令-初學

In [4]:
# 固定教學指令：由函式動態產生（帶入學習階段與模式）
def learning_mode(stage):
    mapping = {
        "初學": "簡單且仔細、步驟拆解、使用生活化比喻與小練習",
        "複習": "深入且精煉、強調重點、分析時間與空間複雜度、列出常見陷阱與邊界條件"
    }
    return mapping.get(stage, "一般解說")

def build_system_instruction(stage, mode=None):
    mode_text = mode or learning_mode(stage)
    return (
        f"你是一位資料結構課程的家教，現在面對一位{stage}資料結構的學生，"
        f"請用繁體中文、採用{mode_text}講解以下單元內容，並適度舉例（需提供解答）。\n"
        "回應結構：\n"
        "1) 核心觀念（3–5點）\n"
        "2) 迷思澄清（2點）\n"
        "3) 例題與詳細解答（至少1題）\n"
        "4) 小測驗（2題，附答案）"
    )

# 教學情緒對應表（回傳 tone 和 style）
def emotion_instruction_map(emotion):
    mapping = {
        "frustrated": {"tone": "溫暖且安撫", "style": "循序漸進、拆解問題"},
        "confused":   {"tone": "溫和且耐心", "style": "舉例對照、比喻解釋"},
        "bored":      {"tone": "活潑且有趣", "style": "加入情境化案例、互動提問"},
        "engaged":    {"tone": "積極且肯定", "style": "深入探討、引導延伸思考"},
        "surprised":  {"tone": "熱情且鼓勵", "style": "延伸趣味點、引入新視角"},
        "joyful":     {"tone": "輕鬆且正向", "style": "融入挑戰題、鼓勵自我探索"},
    }
    return mapping.get(emotion, {"tone": "中性", "style": "一般解釋"})

# ===== 主流程 =====
conversation_history = ""
last_emotion_profile = {"tone": "中性", "style": "一般解釋"}  # 第一單元固定中性
last_emotion = "中性"

learning_stage = "初學"   # 或 "複習"

base_system_instruction = build_system_instruction(learning_stage)

for idx, unit in enumerate(units, start=1):
    print(f"\n=== 單元 {idx}：{unit['unit_title']} ===\n")

    # 第一單元：中性語氣；第二單元起加入情緒的口吻/風格
    emotion_line = "" if idx == 1 else (
        f"你現在應以{last_emotion_profile['tone']}的語氣，並採用{last_emotion_profile['style']}呈現。"
    )

    # === 單元講解 Prompt ===
    lecture_prompt = (
        base_system_instruction + "\n" +
        emotion_line + "\n" +
        "以下是本單元教材，請根據教材進行講解，不要回答其他問題：\n" +
        unit["content"]
    )

    print("🔹 正在生成教學內容...")
    lecture_resp = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=[types.Content(role="user", parts=[types.Part(text=lecture_prompt)])]
    )
    lecture_text = lecture_resp.text
    display(to_markdown(lecture_text))

    # === 問答階段 ===
    print("🔹 進入問答階段")
    while True:
        q = input("學生提問或輸入 next 進入下一單元：")
        if q.strip().lower() == "next":
            break

        tone = last_emotion_profile.get("tone", "中性")
        style = last_emotion_profile.get("style", "一般解釋")

        # === 問題回答 Prompt（與單元講解分開） ===
        qa_prompt = (
            f"你是一位{tone}的資料結構助教，請用{style}的方式回答學生問題。\n"
            "回答時僅能依據以下教材內容作答；如果問題與教材無關，請回答「這個問題與本單元教材無關」。\n"
            f"教材：\n{unit['content']}\n"
            f"學生問題：{q}"
        )

        print("🔹 正在生成答覆...")
        ans_resp = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=[types.Content(role="user", parts=[types.Part(text=qa_prompt)])]
        )
        ans_text = ans_resp.text
        display(to_markdown(ans_text))
        conversation_history += f"\n[單元{idx}] 問：{q}\n答：{ans_text}"

    # 問答結束後輸入新的情緒
    detected_emotion = input("請輸入偵測到的學生情緒：")
    last_emotion_profile = emotion_instruction_map(detected_emotion)
    last_emotion = detected_emotion


=== 單元 1：串列表示法 ===

🔹 正在生成教學內容...


> 好的，這位同學，我們今天要學習的是資料結構中一個很基礎但很重要的概念：**線性串列**。我會用最簡單的方式，一步一步帶你理解它，不用擔心！
> 
> **1) 核心觀念（3–5點）**
> 
> *   **什麼是線性串列？** 想像一下，你去排隊買珍珠奶茶，隊伍裡的人一個接著一個，按照順序排列。這個隊伍，就是一個線性串列。資料結構裡的線性串列，就像這個隊伍一樣，資料一個挨著一個，按照特定的順序排列。
> *   **順序很重要！** 線性串列裡，每個資料都有它的位置，這個位置就是它的「索引」。在隊伍裡，你知道你是第幾個人，就知道輪到你買珍奶的時間大概是什麼時候。線性串列也是一樣，透過索引，你可以快速找到想要的資料。
> *   **常見的操作：** 就像隊伍裡會有人插隊（加入）、有人離開（刪除），線性串列也有一些基本的操作：
>     *   **取出資料：** 知道你是隊伍裡的第幾個人，就能知道你會買到什麼口味的珍奶（假設大家都點一樣的）。
>     *   **計算長度：** 數數隊伍裡有多少人。
>     *   **加入資料：** 有人插隊到隊伍裡的某個位置。
>     *   **刪除資料：** 有人離開隊伍。
> *   **Python 串列的應用：** 在 Python 裡，我們通常用「串列 (List)」來實現線性串列。Python 串列非常好用，你可以把它想像成一個可以裝任何東西的容器，裡面的東西按照順序排列。
> *   **地址的概念：** 想像每個隊員都有一個號碼牌，並且號碼牌有規則地編號，例如從1001開始，每個號碼牌間隔1。那麼要知道第5個人的號碼牌，可以用 1001 + 5 * 1 算出。線性串列中每個元素儲存的位置，也遵守類似的規則。
> 
> **2) 迷思澄清（2點）**
> 
> *   **迷思一：線性串列只能放數字？**  錯！線性串列可以放任何類型的資料，像是數字、文字、甚至是其他的串列。就像你的珍奶隊伍，可以排隊買各種口味的珍奶，不一定要只買同一種。
> *   **迷思二：線性串列的索引從 1 開始？**  不一定！雖然有些地方會從 1 開始算，但 **Python 串列的索引是從 0 開始的！**  這是很重要的一點，一定要記住。所以，隊伍裡的第 1 個人，在 Python 串列裡，索引是 0。
> 
> **3) 例題與詳細解答（至少1題）**
> 
> **題目：**
> 
> 假設我們有一個 Python 串列 `numbers = [10, 20, 30, 40, 50]`。請你取出串列中索引為 2 的元素，並計算這個串列的長度。
> 
> **解答：**
> 
> ```python
> numbers = [10, 20, 30, 40, 50]
> 
> # 取出索引為 2 的元素
> element_at_index_2 = numbers[2]  # 注意：索引從 0 開始，所以索引 2 對應的是 30
> 
> # 計算串列的長度
> list_length = len(numbers)
> 
> print("索引為 2 的元素:", element_at_index_2)  # 輸出：索引為 2 的元素: 30
> print("串列的長度:", list_length)  # 輸出：串列的長度: 5
> ```
> 
> **詳細解釋：**
> 
> 1.  **`numbers[2]`**: 這一行程式碼的意思是，從 `numbers` 這個串列中，取出索引為 2 的元素。因為 Python 串列的索引從 0 開始，所以 `numbers[2]` 實際上取出的是串列中的第三個元素，也就是 30。
> 2.  **`len(numbers)`**: 這一行程式碼的意思是，計算 `numbers` 這個串列的長度。 `len()` 是 Python 的一個內建函數，可以很方便地計算出串列中元素的個數。
> 
> **4) 小測驗（2題，附答案）**
> 
> **題目 1：**
> 
> 假設我們有一個 Python 串列 `fruits = ["apple", "banana", "orange", "grape"]`。如果我們要取出 "banana" 這個元素，應該使用哪個索引？
> 
> A) 0  B) 1  C) 2  D) 3
> 
> **題目 2：**
> 
> 如果我們想要在 Python 串列 `my_list = [1, 2, 4, 5]` 的索引 2 的位置插入數字 3，讓串列變成 `[1, 2, 3, 4, 5]`，你會怎麼做？（提示：Python 串列有 `insert()` 方法可以用。）
> 
> **答案：**
> 
> *   **題目 1：** B) 1 (因為 "apple" 的索引是 0, "banana" 的索引是 1)
> *   **題目 2：** `my_list.insert(2, 3)`
> 
> 希望這些講解對你有幫助！線性串列是資料結構的基礎，多做練習，你會越來越熟練的。如果還有問題，隨時都可以問我喔！


🔹 進入問答階段
學生提問或輸入 next 進入下一單元：next
請輸入偵測到的學生情緒：joyful

=== 單元 2：一維串列 ===

🔹 正在生成教學內容...


> 哈囉！別擔心，資料結構沒有想像中那麼可怕，我們今天來學一個超基礎但很重要的概念：**一維陣列的記憶體位址計算**。 這個聽起來很學術，但其實就像找你家在哪裡一樣簡單！準備好跟我一起探索了嗎？Let's go！🚀
> 
> ### 1) 核心觀念（3–5點）
> 
> 想像一下，你的家是一棟公寓大樓，裡面有很多房間，每個房間都住著不同的人（就像陣列裡的資料）。
> 
> *   **🔑核心觀念 1：陣列就像一排緊密相連的房間。**  每個房間（元素）都住著一個人（資料），而且房間之間沒有空隙，緊緊相鄰。
> *   **🔑核心觀念 2：每個房間都一樣大（佔用相同的空間）。**  假設每個房間都是一樣大小，例如都是單人房 (d)。這個 `d` 代表每個元素在記憶體中佔據的空間大小。
> *   **🔑核心觀念 3：知道第一間房間在哪裡（起始位置），就能算出其他房間在哪裡。**  `a₀` 是個關鍵！它代表第一個房間的門牌號碼（起始位址）。有了它，我們就能推算出其他房間的門牌號碼。想像你朋友告訴你：「我家是這條巷子的第一間，之後每間隔 2 號就是你朋友家」，你一定能找到你朋友家。
> *   **🔑核心觀念 4：房間編號（索引）可以從 0 或其他數字開始。**  有些大樓的房間編號從 0 開始，有些從 1 開始，有些甚至從負數開始！`t` 代表起始索引，告訴我們陣列的「第一間房間」是幾號。
> *   **🔑核心觀念 5：公式是幫你快速找到房間的工具。**  `A(i) = a₀ + (i−t)*d` 這個公式就是我們的尋寶地圖！用 `i` (目標房間號碼)，減去 `t` (起始房間號碼)，再乘以 `d` (每間房間大小)，最後加上 `a₀` (第一間房間的位置)，就能算出目標房間的位置 `A(i)` 了！
> 
> ### 2) 迷思澄清（2點）
> 
> *   **🤔迷思 1：「陣列位址計算很難，要背很多公式。」**  其實不用死背！理解核心觀念，把陣列想像成一排房間，公式只是幫助我們快速計算位置的工具。重點在於了解 `a₀`、`d`、`i` 和 `t` 的意義。
> *   **🤔迷思 2：「`d` 一定是 1。」**  `d` 代表每個元素佔用的空間大小，不一定是 1！例如，如果陣列儲存的是整數 (integer)，`d` 可能會是 4（取決於程式語言和硬體）。
> 
> ### 3) 例題與詳細解答（至少1題）
> 
> **例題：**
> 
> 假設有一個賣飲料的攤販，把他的飲料種類用陣列 `Drinks` 儲存起來。`Drinks` 的編號從 5 開始到 15，也就是 `Drinks(5:15)`。已知 `Drinks(5)` 的位置是 200，而且每種飲料佔用 3 個單位空間（d=3）。請問 `Drinks(10)` 的位置是多少？
> 
> **解答：**
> 
> 1.  **認清角色：**
> 
>     *   `a₀` (起始位置) = 200 (Drinks(5) 的位置)
>     *   `d` (每個元素佔用的空間) = 3
>     *   `i` (目標元素索引) = 10 (想知道 Drinks(10) 的位置)
>     *   `t` (起始索引) = 5 (陣列從 Drinks(5) 開始)
> 2.  **套用公式：**
> 
>     *   `Drinks(i) = a₀ + (i - t) * d`
>     *   `Drinks(10) = 200 + (10 - 5) * 3`
>     *   `Drinks(10) = 200 + (5) * 3`
>     *   `Drinks(10) = 200 + 15`
>     *   `Drinks(10) = 215`
> 3.  **結論：**
> 
>     `Drinks(10)` 的位置是 215。
> 
> ### 4) 小測驗（2題，附答案）
> 
> **💪挑戰題 1：**
> 
> 有一個字串陣列 `Names(-1:5)`，`Names(-1)` 的起始位置是 500，每個字串佔用 10 個單位空間。請問 `Names(3)` 的位置是多少？
> 
> **💪挑戰題 2：**
> 
> 如果一個陣列 `Scores(10:20)`，已知 `Scores(15)` 的位置是 1000，每個元素佔用 4 個單位空間。請問 `Scores(10)` 的位置是多少？（提示：需要稍微反向思考一下喔！）
> 
> ---
> 
> **答案：**
> 
> *   **挑戰題 1 答案：**
> 
>     1.  **認清角色：**
>         *   `a₀` = 500
>         *   `d` = 10
>         *   `i` = 3
>         *   `t` = -1
>     2.  **套用公式：**
>         *   `Names(3) = 500 + (3 - (-1)) * 10`
>         *   `Names(3) = 500 + (4) * 10`
>         *   `Names(3) = 500 + 40`
>         *   `Names(3) = 540`
>     3.  **結論：**
>         `Names(3)` 的位置是 540。
> *   **挑戰題 2 答案：**
> 
>     1.  **先算出 a0:**
>         *   `Scores(15) = a0 + (15-10) * 4 = 1000`
>         *   `a0 + 5 * 4 = 1000`
>         *   `a0 + 20 = 1000`
>         *   `a0 = 980`
>     2.  **再算 Scores(10):**
>         *   `Scores(10) = 980 + (10 - 10) * 4`
>         *   `Scores(10) = 980 + 0`
>         *   `Scores(10) = 980`
>     3.  **結論：**
>         `Scores(10)` 的位置是 980。
> 
> 希望這些例子和測驗能幫助你更好地理解一維陣列的位址計算。記住，資料結構就像組裝樂高一樣，只要掌握了基礎零件的特性，就能創造出各種有趣的結構！下次見囉！👋


🔹 進入問答階段
學生提問或輸入 next 進入下一單元：next
請輸入偵測到的學生情緒：engaged

=== 單元 3：二維串列 ===

🔹 正在生成教學內容...


...

學生提問或輸入 next 進入下一單元：next
請輸入偵測到的學生情緒：confused

=== 單元 6：矩陣相乘 ===

🔹 正在生成教學內容...


> 好的，別擔心，矩陣乘法乍看之下有點複雜，但其實拆解開來，你會發現它很有規律！讓我用簡單的方式，搭配生活化的比喻，一步一步帶你理解。
> 
> **1) 核心觀念**
> 
> *   **核心概念：行與列的對話** 矩陣乘法就像兩個團隊在開會。A團隊的每一列成員要和B團隊的每一行成員進行配對討論，各自貢獻力量，最終共同產出一個新的團隊C。
> *   **維度限制：A說了算，B要配合** A矩陣的 *欄數 (n)* 必須等於B矩陣的 *列數 (n)*，才能進行乘法運算。這就像兩個團隊要合作，人數要對等才行。
> *   **結果：新的矩陣誕生** 相乘後的矩陣C，列數跟A一樣 (*m*)，欄數跟B一樣 (*s*)。所以，如果A是 *m x n*，B是 *n x s*，那C就是 *m x s*。
> *   **個別元素：計算公式**  C矩陣中第 i 列第 j 欄的元素（記作Cij），是A矩陣的第 i 列和B矩陣的第 j 欄對應元素相乘後加總的結果。就像A團隊第i列的成員和B團隊第j行的成員，一個一個的討論合作，總結出的成果。
> *   **順序很重要：先列後行** 計算Cij時，永遠是 A矩陣的第 i 列 *「依序」* 乘以 B矩陣的第 j 欄 *「依序」* 。
> 
> **2) 迷思澄清**
> 
> *   **迷思一：只要兩個矩陣都可以乘？**  
>     *   *真相：* 絕對不是！A矩陣的欄數一定要等於B矩陣的列數才可以。如果A是3x2，B是2x4，就可以相乘。但如果A是3x2，B是3x4，就不能相乘！
> *   **迷思二：AB和BA結果一樣？**
>     *   *真相：*  在大多數情況下，AB和BA的結果是不一樣的！矩陣乘法不具有交換律。就像先跟A團隊合作再跟B團隊合作，跟反過來合作，結果常常不一樣。而且，如果AB可以算出來，BA甚至可能根本不能算（因為維度不符合）。
> 
> **3) 例題與詳細解答**
> 
> **題目：**
> 
> 假設 A =  
> ```
> | 1  2 |
> | 3  4 |
> ```
> 
> B =  
> ```
> | 5  6 |
> | 7  8 |
> ```
> 
> 請計算 AB。
> 
> **解答：**
> 
> 1.  **確認是否可以相乘：**  A是2x2矩陣，B也是2x2矩陣。A的欄數(2)等於B的列數(2)，所以可以相乘。結果矩陣C會是2x2的。
> 
> 2.  **計算C11：**  A的第一列 (1, 2) 乘以 B的第一欄 (5, 7)。
>     C11 = (1 * 5) + (2 * 7) = 5 + 14 = 19
> 
> 3.  **計算C12：** A的第一列 (1, 2) 乘以 B的第二欄 (6, 8)。
>     C12 = (1 * 6) + (2 * 8) = 6 + 16 = 22
> 
> 4.  **計算C21：** A的第二列 (3, 4) 乘以 B的第一欄 (5, 7)。
>     C21 = (3 * 5) + (4 * 7) = 15 + 28 = 43
> 
> 5.  **計算C22：** A的第二列 (3, 4) 乘以 B的第二欄 (6, 8)。
>     C22 = (3 * 6) + (4 * 8) = 18 + 32 = 50
> 
> 6.  **寫出結果：**
> 
> AB =  
> ```
> | 19  22 |
> | 43  50 |
> ```
> 
> **4) 小測驗**
> 
> 1.  假設 A是 3x4 矩陣，B是 4x2 矩陣，那麼 AB 的矩陣維度是多少？ (A) 2x3 (B) 3x2 (C) 4x4 (D) 無法計算
> 2.  假設 A =  
>     ```
>     | 1 |
>     | 2 |
>     ```
>     B = `| 3  4 |`
>     請問 AB = ?
> 
> **答案：**
> 
> 1.  (B) 3x2
> 2.  ```
>     | 3  4 |
>     | 6  8 |
>     ```
> 
> 希望這個講解能幫助你更好地理解矩陣乘法！如果還有任何問題，隨時都可以問我喔！


🔹 進入問答階段
學生提問或輸入 next 進入下一單元：next
請輸入偵測到的學生情緒：focused

=== 單元 7：稀疏矩陣 ===

🔹 正在生成教學內容...


> 好的，沒問題！讓我來一步一步地講解稀疏矩陣這個概念。我會盡量用簡單易懂的方式，搭配生活化的比喻，幫助你理解。
> 
> **1) 核心觀念**
> 
> *   **什麼是稀疏矩陣？**
>     想像一下，你有一張很大的表格，裡面大部分的格子都是空的（填 0）。這種表格，我們就稱它為稀疏矩陣。 就像是你的通訊錄，可能有幾百個聯絡人，但你真正常用的可能只有十幾個。
> *   **為什麼要特別處理？**
>     如果我們真的把所有 0 都存下來，會浪費很多空間。就像你家的倉庫，如果塞滿你根本用不到的東西，那真正需要的東西就沒地方放了。
> *   **簡化的表示方法： (i, j, value)**
>     為了節省空間，我們只記錄「不是 0 」的格子。用 `(i, j, value)` 告訴我們：
>     *   `i`：這個數字在第幾列
>     *   `j`：這個數字在第幾行
>     *   `value`：這個數字是多少
>     就像是餐廳點餐，你只需要告訴服務員「我要第幾號餐，幾份」一樣。
> *   **用二維串列儲存： A(0:n, 1:3)**
>     我們用一個二維串列（你可以想像成一個表格）來儲存這些非零的值。
>     *   `A(0:n, 1:3)` 表示這個表格有 `n+1` 列，3 行。
>     *   每一列代表一個非零元素，包含它的列號、行號和值。
>     *   第一行放列號 (`i`)，第二行放行號 (`j`)，第三行放值 (`value`)。
> *   **節省空間的優勢：**
>     相較於儲存整個矩陣，這種方式在稀疏矩陣中能夠大幅減少記憶體的使用。
> 
> **2) 迷思澄清**
> 
> *   **迷思一：一定要超過一半是 0 才是稀疏矩陣？**
>     教材說「一般而言，大於 1/2 個就可稱之」，這只是一個參考。實際上，沒有硬性規定。重點是，如果 0 很多，用這種簡化方法才能省下空間。
> *   **迷思二：這種方法只適用於很大的矩陣？**
>     雖然在大矩陣中效果更明顯，但只要 0 的比例夠高，即使是小矩陣也可以使用這種方法。
> 
> **3) 例題與詳細解答**
> 
> 假設我們有一個 4x4 的矩陣：
> 
> ```
> 0  0  3  0
> 0  0  0  0
> 2  0  0  0
> 0  0  0  5
> ```
> 
> 用 `(i, j, value)` 的方式表示，並儲存到 `A(0:n, 1:3)` 的二維串列中。
> 
> **解答：**
> 
> 1.  **找出所有非零元素：**
>     *   (0, 2, 3)  （第 0 列，第 2 行，值是 3）
>     *   (2, 0, 2)  （第 2 列，第 0 行，值是 2）
>     *   (3, 3, 5)  （第 3 列，第 3 行，值是 5）
> 2.  **建立二維串列 A：**
>     因為有 3 個非零元素，所以 `n = 2`，串列 `A` 是 `A(0:2, 1:3)`
> 
> ```
> A = [
>     [0, 2, 3],  # (0, 2, 3)
>     [2, 0, 2],  # (2, 0, 2)
>     [3, 3, 5]   # (3, 3, 5)
> ]
> ```
> 
> **4) 小測驗**
> 
> 1.  如果一個 10x10 的矩陣，只有 5 個元素不是 0，用 `(i, j, value)` 的方法來表示，我們需要儲存多少個數字（包含列號、行號和值）？
> 2.  為什麼要使用稀疏矩陣的表示方法？請簡述其優點。
> 
> **答案：**
> 
> 1.  15 個 (因為 5 個非零元素 * 每個元素需要 3 個數字來儲存)
> 2.  優點：節省儲存空間，特別是當矩陣中大部分元素為 0 時。
> 
> 希望這個講解對你有幫助！如果還有任何問題，隨時都可以問我。


🔹 進入問答階段
學生提問或輸入 next 進入下一單元：next
請輸入偵測到的學生情緒：surprised


有指令-複習

In [ ]:
# 固定教學指令：由函式動態產生（帶入學習階段與模式）
def learning_mode(stage):
    mapping = {
        "初學": "簡單且仔細、步驟拆解、使用生活化比喻與小練習",
        "複習": "深入且精煉、強調重點、分析時間與空間複雜度、列出常見陷阱與邊界條件"
    }
    return mapping.get(stage, "一般解說")

def build_system_instruction(stage, mode=None):
    mode_text = mode or learning_mode(stage)
    return (
        f"你是一位資料結構課程的家教，現在面對一位{stage}資料結構的學生，"
        f"請用繁體中文、採用{mode_text}講解以下單元內容，並適度舉例（需提供解答）。\n"
        "回應結構：\n"
        "1) 核心觀念（3–5點）\n"
        "2) 迷思澄清（2點）\n"
        "3) 例題與詳細解答（至少1題）\n"
        "4) 小測驗（2題，附答案）"
    )

# 教學情緒對應表（回傳 tone 和 style）
def emotion_instruction_map(emotion):
    mapping = {
        "frustrated": {"tone": "溫暖且安撫", "style": "循序漸進、拆解問題"},
        "confused":   {"tone": "溫和且耐心", "style": "舉例對照、比喻解釋"},
        "bored":      {"tone": "活潑且有趣", "style": "加入情境化案例、互動提問"},
        "engaged":    {"tone": "積極且肯定", "style": "深入探討、引導延伸思考"},
        "surprised":  {"tone": "熱情且鼓勵", "style": "延伸趣味點、引入新視角"},
        "joyful":     {"tone": "輕鬆且正向", "style": "融入挑戰題、鼓勵自我探索"},
    }
    return mapping.get(emotion, {"tone": "中性", "style": "一般解釋"})

# ===== 主流程 =====
conversation_history = ""
last_emotion_profile = {"tone": "中性", "style": "一般解釋"}  # 第一單元固定中性
last_emotion = "中性"

learning_stage = "複習"   # 或 "初學"

base_system_instruction = build_system_instruction(learning_stage)

for idx, unit in enumerate(units, start=1):
    print(f"\n=== 單元 {idx}：{unit['unit_title']} ===\n")

    # 第一單元：中性語氣；第二單元起加入情緒的口吻/風格
    emotion_line = "" if idx == 1 else (
        f"你現在應以{last_emotion_profile['tone']}的語氣，並採用{last_emotion_profile['style']}呈現。"
    )

    # === 單元講解 Prompt ===
    lecture_prompt = (
        base_system_instruction + "\n" +
        emotion_line + "\n" +
        "以下是本單元教材，請根據教材進行講解，不要回答其他問題：\n" +
        unit["content"]
    )

    print("🔹 正在生成教學內容...")
    lecture_resp = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=[types.Content(role="user", parts=[types.Part(text=lecture_prompt)])]
    )
    lecture_text = lecture_resp.text
    display(to_markdown(lecture_text))

    # === 問答階段 ===
    print("🔹 進入問答階段")
    while True:
        q = input("學生提問或輸入 next 進入下一單元：")
        if q.strip().lower() == "next":
            break

        tone = last_emotion_profile.get("tone", "中性")
        style = last_emotion_profile.get("style", "一般解釋")

        # === 問題回答 Prompt（與單元講解分開） ===
        qa_prompt = (
            f"你是一位{tone}的資料結構助教，請用{style}的方式回答學生問題。\n"
            "回答時僅能依據以下教材內容作答；如果問題與教材無關，請回答「這個問題與本單元教材無關」。\n"
            f"教材：\n{unit['content']}\n"
            f"學生問題：{q}"
        )

        print("🔹 正在生成答覆...")
        ans_resp = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=[types.Content(role="user", parts=[types.Part(text=qa_prompt)])]
        )
        ans_text = ans_resp.text
        display(to_markdown(ans_text))
        conversation_history += f"\n[單元{idx}] 問：{q}\n答：{ans_text}"

    # 問答結束後輸入新的情緒
    detected_emotion = input("請輸入偵測到的學生情緒：")
    last_emotion_profile = emotion_instruction_map(detected_emotion)
    last_emotion = detected_emotion


=== 單元 1：串列基本概念與表示法 ===

🔹 正在生成教學內容...


> 好的，讓我來為你深入講解串列 (List) 這個資料結構。
> 
> **1) 核心觀念**
> 
> *   **線性序列：** 串列的核心在於其線性排列的特性。元素之間存在明顯的前後關係，每個元素都有其特定的位置。
> *   **索引存取：** 串列中的元素可以通過索引值快速存取。這是串列最重要的特性之一，也是其與集合 (Set) 等資料結構的主要區別。
> *   **動態大小：** 在許多程式語言 (例如 Python) 中，串列的大小是動態的，可以根據需要增加或刪除元素，而不需要預先宣告固定的大小。（但底層實作可能涉及記憶體重新配置，影響效能）
> *   **資料類型彈性：** 許多高階語言的串列允許儲存不同資料類型的元素，例如 Python 的 list 可以同時包含整數、字串和物件。
> *   **常見操作：** 串列支援一系列基本操作，例如插入、刪除、搜尋、遍歷等，這些操作的時間複雜度會根據具體實現而有所不同。
> 
> **2) 迷思澄清**
> 
> *   **迷思一：串列的存取時間複雜度永遠是 O(1)。**
>     *   **澄清：** 雖然通過索引存取元素在理想情況下是 O(1)，但在 Python 等動態語言中，底層實現可能會因資料類型或記憶體配置等因素導致存取時間略有變化。更重要的是，插入和刪除操作通常不是 O(1)，因為它們可能需要移動其他元素。
> *   **迷思二：串列只能儲存相同資料類型的元素。**
>     *   **澄清：** 雖然在某些語言 (例如 C++ 中的 `std::vector` 如果不使用 `variant`) 中，串列需要儲存相同類型的元素，但在 Python 等動態類型語言中，串列可以儲存不同類型的元素。然而，儲存不同類型的元素可能會影響效能，因為需要額外的類型檢查。
> 
> **3) 例題與詳細解答**
> 
> **題目：** 給定一個 Python 串列 `nums = [1, 2, 3, 4, 5]`，請編寫程式碼實現在指定位置插入一個新的元素，並刪除指定位置的元素。
>     *   分別計算在串列頭部插入元素、在串列尾部插入元素的時間複雜度。
>     *   分別計算在串列頭部刪除元素、在串列尾部刪除元素的時間複雜度。
> 
> **解答：**
> 
> ```python
> nums = [1, 2, 3, 4, 5]
> 
> # 在索引 2 插入元素 10
> nums.insert(2, 10)
> print(f"在索引2插入元素10後的串列: {nums}")  # 輸出：[1, 2, 10, 3, 4, 5]
> 
> # 刪除索引 3 的元素
> del nums[3]
> print(f"刪除索引3的元素後的串列: {nums}")  # 輸出：[1, 2, 10, 4, 5]
> 
> # 時間複雜度分析：
> # 1. 在串列頭部插入元素：O(n)，因為需要將所有後續元素後移。
> # 2. 在串列尾部插入元素：O(1) (平均情況)，因為通常不需要移動元素。
> # 3. 在串列頭部刪除元素：O(n)，因為需要將所有後續元素前移。
> # 4. 在串列尾部刪除元素：O(1)，因為不需要移動元素。
> 
> # 空間複雜度分析：
> # insert 和 del 操作通常不需要額外的空間，所以空間複雜度是 O(1)。
> ```
> 
> **4) 小測驗**
> 
> 1.  給定一個Python串列 `my_list = [5, 2, 8, 1, 9]`，如何使用Python程式碼找到串列中的最大值？寫出程式碼，並分析時間複雜度。
> 2.  假設你需要頻繁地在串列的頭部進行插入和刪除操作，使用 Python 的 `list` 是否合適？如果不合適，可以考慮使用哪種資料結構來替代？為什麼？
> 
> **答案：**
> 
> 1.  ```python
>     my_list = [5, 2, 8, 1, 9]
>     max_value = max(my_list)
>     print(f"串列中的最大值是：{max_value}") # 輸出：9
>     # 時間複雜度：O(n)，其中 n 是串列的長度。
>     ```
> 2.  不合適。Python 的 `list` 在頭部插入和刪除的複雜度為 O(n)，因為需要移動元素。更適合的資料結構是 `collections.deque` (雙端佇列)，它在頭部和尾部插入和刪除的時間複雜度都是 O(1)。這是因為 `deque` 通常使用雙向鏈結串列或類似的結構來實現，允許高效的頭尾操作。
> 
> 希望這個講解能幫助你更好地理解串列的相關知識！ 記住，資料結構的選擇取決於具體應用場景的需求。


🔹 進入問答階段
學生提問或輸入 next 進入下一單元：next
請輸入偵測到的學生情緒：joyful

=== 單元 2：一維串列 ===

🔹 正在生成教學內容...


> 哈囉！很高興能跟你一起複習資料結構，今天要攻克的是一維陣列這個基本但重要的概念。別擔心，只要掌握核心觀念，加上一些練習，就能輕鬆搞定！準備好迎接挑戰了嗎？Let's go!
> 
> **1) 核心觀念（3–5點）**
> 
> *   **連續記憶體空間：** 一維陣列的關鍵特性在於它在記憶體中佔用的是一段連續的空間。這使得我們可以通過簡單的數學公式快速計算出任意元素的位址。想像成一排緊密相連的房間，每個房間都住著陣列的一個元素。
> *   **位址計算公式的意義：** 位址計算公式的核心是找到目標元素相對於陣列起點的偏移量。`A(i) = a₀ + (i-t)*d` 這個公式中的 `a₀` 是陣列的起始位址，`i` 是目標元素的索引，`t` 是起始索引 (如果不是從0開始)，`d` 是每個元素佔用的空間大小。簡單來說，就是「起點 + 偏移量」。
> *   **索引（Index）的角色：** 索引就像是房間的門牌號碼，它告訴我們目標元素在陣列中的位置。理解索引如何與記憶體位址聯繫，是掌握陣列的關鍵。
> *   **空間複雜度：** 一維陣列的空間複雜度是 O(n)，其中 n 是陣列的長度。因為需要儲存所有元素。
> *   **時間複雜度：**
>     *   **存取（Access）：** 存取陣列中特定索引的元素，時間複雜度是 O(1)。這是因為可以直接通過位址計算公式找到元素的位址。
>     *   **搜尋（Search）：** 如果要搜尋陣列中是否存在特定元素，最壞情況下需要遍歷整個陣列，時間複雜度是 O(n)。
> 
> **2) 迷思澄清（2點）**
> 
> *   **迷思一：陣列只能從0開始。** 這是初學者常犯的錯誤。陣列的起始索引可以根據需求設定，例如 `-3`, `1`, 甚至其他任意整數。重點在於理解位址計算公式中的 `t` 代表起始索引。
> *   **迷思二：陣列大小不能改變。** 在某些程式語言 (例如 C/C++) 中，陣列的大小在宣告時就必須確定，且不能動態改變。但在其他語言 (例如 Python 的 list) 中，陣列的大小是可以動態調整的。這是不同語言特性造成的差異。
> 
> **3) 例題與詳細解答（至少1題）**
> 
> **題目：** 假設有一個一維陣列 `B(-5:5)`，已知 `B(-2)` 的位址是 1000，每個元素佔用 4 個位元組的空間。請計算 `B(3)` 的位址。
> 
> **解答：**
> 
> 1.  **提取資訊：**
>     *   起始索引 `t = -5`
>     *   `B(-2)` 的位址 `a₀ + (-2 - (-5)) * d = 1000`
>     *   每個元素大小 `d = 4`
>     *   目標索引 `i = 3`
> 
> 2.  **計算起始位址 a₀：**
>     *   `a₀ + (-2 + 5) * 4 = 1000`
>     *   `a₀ + 3 * 4 = 1000`
>     *   `a₀ + 12 = 1000`
>     *   `a₀ = 988`
> 
> 3.  **計算 B(3) 的位址：**
>     *   `B(3) = a₀ + (3 - (-5)) * d`
>     *   `B(3) = 988 + (3 + 5) * 4`
>     *   `B(3) = 988 + 8 * 4`
>     *   `B(3) = 988 + 32`
>     *   `B(3) = 1020`
> 
> **答案：** `B(3)` 的位址是 1020。
> 
> **4) 小測驗（2題，附答案）**
> 
> 1.  **挑戰題：** 一個一維陣列 `C(10:20)`，如果 `C(12)` 的位址是 200，每個元素佔用 2 個位元組，那麼 `C(18)` 的位址是多少？
> 
>     *   (A) 212
>     *   (B) 216
>     *   (C) 220
>     *   (D) 224
> 
> 2.  假設一維陣列 `D(0:9)`，起始位址為 500，每個元素佔用 8 個位元組，請問存取 `D(5)` 的時間複雜度是多少？
> 
>     *   (A) O(1)
>     *   (B) O(log n)
>     *   (C) O(n)
>     *   (D) O(n log n)
> 
> **答案：**
> 
> 1.  (A) 212
> 2.  (A) O(1)
> 
> 希望這次的複習對你有所幫助！記住，理解概念並多加練習，就能輕鬆掌握一維陣列。加油！


🔹 進入問答階段
學生提問或輸入 next 進入下一單元：請講解小測驗的第一題
🔹 正在生成答覆...


...

學生提問或輸入 next 進入下一單元：請問測驗一的答案是15嗎
🔹 正在生成答覆...


> 同學你好，
> 
> 我理解你可能對於測驗一的答案有些疑問，但很抱歉，我無法直接回答你測驗一的答案是否為15。
> 
> 本單元教材主要講解的是如何使用Python建立一維、二維和三維串列，以及如何理解它們的索引和元素個數。
> 
> 教材中提供了以下範例：
> 
> *   **一維串列:** `A = [0] * 20`  (有20個元素，索引從 A[0] 到 A[19])
> *   **二維串列:** `A = [[0] * 10 for rows in range(20)]` (有20列、10行，索引從 A[0][0] 到 A[19][9])
> *   **三維串列:** `A = [[[0 for z in range(10)] for x in range(20)] for y in range(30)]` (有30個"二維串列"，每個"二維串列"有20列，每列有10個元素)
> 
> 如果你能提供測驗一的題目內容，我可以試著根據教材中的知識，幫助你分析題目，讓你更了解如何從題目中提取資訊，並運用串列的概念來解決問題。

學生提問或輸入 next 進入下一單元："如果我定義了一個一維串列 B = [1] * 15，那麼 B 串列總共有幾個元素？最後一個元素的索引是多少?"請問這題的答案是15嗎
🔹 正在生成答覆...


> 同學你好，這個問題問得很好！
> 
> 根據我們教材的內容，當我們定義一個一維串列 `B = [1] * 15`，這表示 `B` 串列裡面會有 15 個元素。
> 
> 就像 `A = [0] * 20` 代表 `A` 串列有 20 個元素一樣，`B = [1] * 15` 就代表 `B` 串列有 15 個元素。
> 
> 至於最後一個元素的索引，就像 `A` 串列的索引是從 `A[0]` 到 `A[19]`，`B` 串列的索引會是從 `B[0]` 開始，那最後一個元素就會是 `B[14]` 喔！
> 
> 所以，你的答案前半部分是對的，`B` 串列總共有 15 個元素，但最後一個元素的索引是 14，不是 15 喔。要記得索引是從 0 開始的。


KeyboardInterrupt: Interrupted by user